In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_train.csv')
test = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_test.csv')


X_train = train.iloc[:,1:]
Y_train = train.iloc[:,0]
X_test = test.iloc[:,1:]
Y_test = test.iloc[:,0]



In [ ]:
X_train.shape,Y_train.shape

In [ ]:
X_train = np.array(X_train) 
X_test = np.array(X_test)


In [ ]:

X_train = X_train/ 255
X_test = X_test/ 255

In [ ]:
X_train =X_train.reshape(-1,28,28,1)
X_test =X_test.reshape(-1,28,28,1)


In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.fit_transform(Y_test)


In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_valid, Y_train,Y_valid = train_test_split(X_train , Y_train, test_size=0.33, random_state=144)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPooling2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.regularizers import l2,l1
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import Activation, Convolution2D, Dropout, Conv2D,MaxPool2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D

from keras.layers import SeparableConv2D

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(28,28,1),filters=20,kernel_size=(3,3),padding="same", activation="relu"))
model.add(keras.layers.Dropout(0.1))


model.add(MaxPool2D(2,2))
model.add(Conv2D(32 , (3,3)  , activation = 'relu' ))
model.add(MaxPool2D(2,2 ))
model.add(keras.layers.Dropout(0.1))

model.add(Conv2D(64 , (3,3)  , activation = 'relu' ))
model.add(MaxPool2D((2,2) ))
model.add(keras.layers.Dropout(0.1))

model.add(Conv2D(128 , (1,1)  , activation = 'relu' ))
model.add(MaxPool2D((2,2) ))
model.add(keras.layers.Dropout(0.1))



model.add(Flatten())
model.add(Dense(units=64,activation="relu"))
model.add(keras.layers.Dropout(0.1))

model.add(Dense(units = 24 , activation = 'softmax'))




In [ ]:
learning_rate = 1e-3
lr_decay = 1e-6
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
adam = optimizers.Adam(lr=0.002)

model.compile(loss='categorical_crossentropy', optimizer=adam,metrics=['accuracy'])

history = model.fit(X_train, Y_train , epochs=60,batch_size =524 , verbose =1,validation_data=(X_valid,Y_valid))

In [ ]:
testModel = model.evaluate(X_test,Y_test)
print("Acuarcy = %.2f%%"%(testModel[1]*100))
print("Loss = %.2f%%"%(testModel[0]*100))
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:


predicted_classes = model.predict_classes(X_test)
rounded_labels=np.argmax(Y_test, axis=1)
confusionMatrix = confusion_matrix(rounded_labels, predicted_classes)
confusionMatrix = pd.DataFrame(confusionMatrix , index = [i for i in range(25) if i != 9] , columns = [i for i in range(25) if i != 9])
plt.figure(figsize = (15,15))
sns.heatmap(confusionMatrix,cmap= "OrRd_r", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')

In [ ]:
classes = ["Class " + str(i) for i in range(25) if i != 9]
print(classification_report(rounded_labels, predicted_classes, target_names = classes))